# Data Preparation

## Packages

In [10]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool


import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [11]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [12]:
def chunk_ticks(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []

    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))

    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    
    return(temp_df)

## Parameters

In [13]:
#pd.DataFrame(np.random.rand(14,4), columns=['a', 'b', 'c', 'd'])

year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')


number_of_ticks = 10
pip_diff = 0.00001
rsi_window = 10
rs_max = 1e6
window_s_ma = 10
window_l_ma = 100

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [40]:
%%time
#df = pd.read_csv(source_file_path, nrows=10000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 27 s


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


## Data manipulation

In [13]:
%%time
df = chunk_ticks(df, number_of_ticks)

df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')
df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████████████████████████████████████████| 2604878/2604878 [14:52<00:00, 2917.38it/s]


Records : 2604878
Wall time: 15min 7s


,tick_avg,spread_avg
0,1.201194,0.000368
1,1.200771,0.000397
2,1.200578,0.000756
3,1.200736,0.000652
4,1.200714,0.000800


In [15]:
%%time
df['tick_diff'] = df['tick_avg'].diff()

# Moving Averages ------------------
df['sema'] = df['tick_avg'].ewm(span=window_s_ma).mean()
df['lema'] = df['tick_avg'].ewm(span=window_l_ma).mean()
df['ema_diff'] = df['sema'] - df['tick_avg']


# RSI -----------------------------
df['diff'] = df['sema'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=window_s_ma).progress_apply(get_slope_s)
df['long_sema_slope'] = df['sema'].rolling(window=window_l_ma).progress_apply(get_slope_l)

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > pip_diff] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -pip_diff] = 'decrease'

# Remove NaNs ------------------------
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

2604869it [12:34, 3450.81it/s]
2604779it [16:57, 2561.12it/s]


Total records : 2604779
Wall time: 29min 35s


## Write data to csv

In [16]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 2min 24s


## Print Report

In [17]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))

           counts  percentage
same      1339163   51.411770
decrease   634933   24.375696
increase   630683   24.212534


In [18]:
df.head(5)

,tick_avg,spread_avg,tick_diff,sema,lema,ema_diff,diff,gain,loss,avg_gain,avg_loss,rs,rsi,small_sema_slope,long_sema_slope,direction
0,1.201482,0.000157,1.000000e-05,1.201457,1.201171,-0.000025,0.000006,0.000006,0.0,0.000002,0.000006,0.260110,20.641870,-24.468222,36.156279,same
1,1.201481,0.000178,-5.000000e-07,1.201461,1.201178,-0.000020,0.000004,0.000004,0.0,0.000002,0.000006,0.377234,27.390691,-11.820742,37.596694,same
2,1.201483,0.000176,2.000000e-06,1.201465,1.201185,-0.000018,0.000004,0.000004,0.0,0.000003,0.000004,0.624199,38.431208,-0.274442,38.488405,same
3,1.201496,0.000173,1.350000e-05,1.201471,1.201192,-0.000026,0.000006,0.000006,0.0,0.000003,0.000003,1.013613,50.338012,10.405255,39.028280,same
4,1.201492,0.000158,-4.500000e-06,1.201475,1.201199,-0.000017,0.000004,0.000004,0.0,0.000003,0.000002,1.608876,61.669312,18.440327,39.492326,same
